# Heuristics Tests for the Traveling Tournament Problem (TTP)
This notebook tests several heuristics, meta-heuristics, and hybrid methods to solve (usually large) instances of the conventional TTP.

In [1]:
# Import libraries

import copy
import json
import math
import networkx as nx
import numpy as np
import pandas as pd
import random as rd
import xmltodict

## Parse Data

In this section, we parse XML files downloaded from the following site: https://robinxval.ugent.be/RobinX/travelRepo.php

Each XML has information pertaining a specific instance of the TTP. These can be converted to JSON format, and then to Python dictionaries, for easier management of the data within. 

In [2]:
# Convert XML data to JSON / Python dictionary

nl4 = xmltodict.parse(open('NL4.xml').read())
# nl4 = json.dumps(nl4)
print(nl4)

{'Instance': {'MetaData': {'InstanceName': 'NL4', 'DataType': 'A', 'Contributor': 'Easton, Nemhauser, and Trick', 'Date': {'@day': '0', '@month': '0', '@year': '2001'}, 'Country': None, 'Remarks': 'Based on National Hockey League', 'Lowerbound': {'@infeasibility': '0', '@objective': '0'}}, 'Structure': {'Format': {'@leagueIds': '0', 'numberRoundRobin': '2', 'compactness': 'C'}, 'AdditionalGames': None}, 'ObjectiveFunction': {'Objective': 'TR'}, 'Data': {'Distances': {'distance': [{'@dist': '0', '@team1': '0', '@team2': '0'}, {'@dist': '745', '@team1': '0', '@team2': '1'}, {'@dist': '665', '@team1': '0', '@team2': '2'}, {'@dist': '929', '@team1': '0', '@team2': '3'}, {'@dist': '745', '@team1': '1', '@team2': '0'}, {'@dist': '0', '@team1': '1', '@team2': '1'}, {'@dist': '80', '@team1': '1', '@team2': '2'}, {'@dist': '337', '@team1': '1', '@team2': '3'}, {'@dist': '665', '@team1': '2', '@team2': '0'}, {'@dist': '80', '@team1': '2', '@team2': '1'}, {'@dist': '0', '@team1': '2', '@team2': '

In [3]:
# Obtain relevant data

# Distances
distances = dict()
for distance in nl4['Instance']['Data']['Distances']['distance']:
    distances[(int(distance['@team1']), int(distance['@team2']))] = int(distance['@dist'])
print(distances)

# Teams
teams = dict()
for team in nl4['Instance']['Resources']['Teams']['team']:
    teams[int(team['@id'])] = team['@name']
print(teams)

{(0, 0): 0, (0, 1): 745, (0, 2): 665, (0, 3): 929, (1, 0): 745, (1, 1): 0, (1, 2): 80, (1, 3): 337, (2, 0): 665, (2, 1): 80, (2, 2): 0, (2, 3): 380, (3, 0): 929, (3, 1): 337, (3, 2): 380, (3, 3): 0}
{0: 'ATL', 1: 'NYM', 2: 'PHI', 3: 'MON'}


## Greedy Algorithm
We use this algorithm to find an initial feasible solution to the TTP (assuming every team can directly travel to every other team's home).

In [6]:
# Early versions

def greedy_algorithm_ernesto(n: int):
    """
    Greedy algorithm for TTP data.
    Input:
        - n: int --> number of participating teams (even number)
    Output:
        - initial_sol: dict --> initial solution for double round-robin, where keys are rounds and values are lists of matchups
            Note: each matchup is represented as a tuple of the form (team1, team2), such that the match is played in team1's home
    """
    
    # Initialize solution dictionary
    initial_sol = dict()
    for round_num in range(n - 1):
        initial_sol[round_num] = []
    
    # Create list of lexopgraphical matchups    
    lexographical_matchups = [(i, j) for i in range(n - 1) for j in range(i + 1, n)]
    
    # Fill solution dictionary with matchups
    matchup_clusters_added = 0
    while matchup_clusters_added < n/2:
        new_matchups_added = 0
        round_num = 0
        while new_matchups_added < n - 1:
            if initial_sol[round_num] == []:    # Check if round is empty and add matchup
                initial_sol[round_num].append(lexographical_matchups[0])
                lexographical_matchups.pop(0)
                new_matchups_added += 1
            else:   # Check for team conflicts in round
                visited_nums = []
                for round_match in initial_sol[round_num]:
                    visited_nums.append(round_match[0])
                    visited_nums.append(round_match[1])
                if lexographical_matchups[0][0] not in visited_nums and lexographical_matchups[0][1] not in visited_nums:   # Condition for adding matchup if round isn't empty
                    initial_sol[round_num].append(lexographical_matchups[0])
                    lexographical_matchups.pop(0)
                    new_matchups_added += 1
            round_num += 1
            round_num %= n - 1  # Reset round number after reaching last round
        matchup_clusters_added += 1
    
    # Add mirrored results to complete the double round-robin
    for round_num in range(n - 1):
        initial_sol[n - 1 + round_num] = []
        for matchup in initial_sol[round_num]:
            initial_sol[n - 1 + round_num].append((matchup[1], matchup[0]))
    
    return initial_sol 

def greedy_algorithm_wessel(n: int) -> dict:
    """
    Greedy algorithm for producing a compact single round-robin schedule for n teams.
    This schedule forms the first half of a double round-robin (DRR) solution.

    Input:
        - n: int --> number of participating teams (must be even for a proper round-robin)
    Output:
        - initial_srr_sol: dict --> initial solution for the single round-robin (SRR), 
                                     where keys are round indices (0 to n-2) and values 
                                     are lists of matchups (tuples of teams).
    """
    num_rounds = n - 1
    initial_srr_sol = {i: [] for i in range(num_rounds)}

    lexographical_matchups = [(i, j) for i in range(n) for j in range(i + 1, n)]
    
    current_round = 0
    while lexographical_matchups:
        matchup = lexographical_matchups.pop(0) 
        team_a, team_b = matchup
        match_placed = False
        
        for _ in range(num_rounds):
            round_index = current_round % num_rounds
        
            # Get all teams currently scheduled in the target round
            teams_in_round = set()
            for existing_match in initial_srr_sol[round_index]:
                teams_in_round.add(existing_match[0])
                teams_in_round.add(existing_match[1])

            # Check for clash
            if team_a not in teams_in_round and team_b not in teams_in_round:
                initial_srr_sol[round_index].append(matchup)
                match_placed = True
                current_round = (round_index + 1) % num_rounds
                break
            else:    
                current_round = (current_round + 1) % num_rounds
    
    #Make the drr
    initial_drr_sol = initial_srr_sol.copy()
    for i in range(num_rounds):
        initial_drr_sol[num_rounds + i] = [matchup[::-1] for matchup in initial_srr_sol[i]]

    return initial_drr_sol

In [4]:
# Refined version

def greedy_algorithm(n: int):
    """
    Greedy algorithm for TTP data.
    Input:
        - n: int --> number of participating teams (must be even for a feasible round-robin)
    Output:
        - initial_sol: dict --> initial solution for double round-robin, where keys are rounds and values are lists of matchups
            Notes:
                - each matchup is represented as a tuple of the form (home_team, away_team)
                - there are 2*(n-1) total rounds in a double round-robin, where n/2 matchups are played in each round
    """
    
    # Initialize solution dictionary with single round-robin matchups
    initial_sol = {i: [] for i in range(n - 1)}
    
    # Create list of lexopgraphical matchups    
    lexographical_matchups = [(i, j) for i in range(n - 1) for j in range(i + 1, n)]
    
    # Fill solution dictionary with matchups
    current_round = 0
    while lexographical_matchups:
        home_team, away_team = lexographical_matchups[0]
        
        # Get all teams currently scheduled in the target round
        teams_in_round = set()
        for existing_match in initial_sol[current_round]:
            teams_in_round.add(existing_match[0])
            teams_in_round.add(existing_match[1])
        
        # Check for repeated teams in round
        if home_team not in teams_in_round and away_team not in teams_in_round:
            initial_sol[current_round].append((home_team, away_team))   # Add matchup to round
            lexographical_matchups.pop(0)   # Remove matchup from lexographical list
        current_round = (current_round + 1) % (n - 1)   # Move to next round (or first round if at last round)
    
    # Add mirrored results to complete the double round-robin
    for round_num in range(n - 1):
        initial_sol[round_num + n - 1] = [matchup[::-1] for matchup in initial_sol[round_num]]
    
    return initial_sol 

In [5]:
# Tests

n = 4
lexographical_matchups = [(i, j) for i in range(n - 1) for j in range(i + 1, n)]
print(lexographical_matchups)

initial_sol = greedy_algorithm(n)
print(initial_sol)

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
{0: [(0, 1), (2, 3)], 1: [(0, 2), (1, 3)], 2: [(0, 3), (1, 2)], 3: [(1, 0), (3, 2)], 4: [(2, 0), (3, 1)], 5: [(3, 0), (2, 1)]}


## Simulated Annealing / Tabu Search

We have a solution space given by coloring schemes of a graph G, such that:
- each vertex v in G represents a possible matchup (for n teams in a double round robin, there are n*(n-1) possible matchups – excluding impossible matchups of a team competing against itself) 
- each edge e in G is placed between vertices that share at least one team in their matchups
- we use 2*(n - 1) colors for the vertices of G, such that neighboring vertices cannot share the same color
    - here, each color represents the matchups within each of the 2*(n-1) rounds in a double round-robin

To travel along this solution space, we use the neighboring moves described in the paper "An Efficient Simulated Annealing Approach to the Travelling Tournament Problem". Each move is chosen at random* based on the possible choices, and may represent an improving or non-improving difference of a solution's value compared to its predecesor. We determine whether to make the move or not with the simulated annealing meta-heuristic, which proposes a temperature T such that:
- if the value of the new solution is improving on our optimization function --> we make the move
- if the value of the new solution is non-improving on our optimization function --> we have a probability of exp(-delta/T), where delta is the difference in values of our previous and new solutions
- the temperature parameter T decreases over the number of iterations (implying that the probability of choosing non-improving moves decreases over time). The rate of decrease is given by a fixed 'cooling parameter'. 

*more info on this in the paper

### Auxiliary Functions

In [9]:
# Functions for neighboring moves

def swap_rounds(sol: dict, round_1: int, round_2: int):
    """
    Swap two rounds in a solution.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - round1, round2: int --> rounds to be swapped
    Output:
        - new_sol: dict --> new solution where round1 is changed to round2, and viceversa
    """
    
    new_sol = copy.deepcopy(sol)
    new_sol[round_1] = sol[round_2]
    new_sol[round_2] = sol[round_1]
    
    return new_sol

def swap_homes(sol: dict, team_1: int, team_2: int):
    """
    Swap the homes of two teams in a solution.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - team_1, team_2: int --> teams to be swapped
    Output:
        - new_sol: dict --> new solution where (team_1, team_2) is changed to (team_2, team_1), and viceversa
    """
    
    new_sol = copy.deepcopy(sol)
    for round in sol:
        if (team_1, team_2) in sol[round]:
            new_sol[round].remove((team_1, team_2))
            new_sol[round].append((team_2, team_1))
        elif (team_2, team_1) in sol[round]:
            new_sol[round].remove((team_2, team_1))
            new_sol[round].append((team_1, team_2))
    
    return new_sol

def swap_schedules(sol: dict, team_1: int, team_2: int):
    """
    Swap the schedules of two teams in a solution, except for the games they play against each other.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - team_1, team_2: int --> teams with schedules to be swapped
    Output:
        - new_sol: dict --> new solution where the positions of team_1 and team_2 are swapped in each round, except when they play against each other
    """
    
    new_sol = copy.deepcopy(sol)
    for round in sol:
        if ((team_1, team_2) not in sol[round]) and ((team_2, team_1) not in sol[round]):
            updated_matches = []
            for team_x, team_y in sol[round]:
                if team_x == team_1:
                    updated_matches.append((team_2, team_y))
                elif team_x == team_2:
                    updated_matches.append((team_1, team_y))
                elif team_y == team_1:
                    updated_matches.append((team_x, team_2))
                elif team_y == team_2:
                    updated_matches.append((team_x, team_1))
                else:
                    updated_matches.append((team_x, team_y))
            new_sol[round] = updated_matches
    
    return new_sol

def swap_rounds_partial(sol: dict, team: int, round_1: int, round_2: int):
    """
    Docstring
    """
    
    new_sol = copy.deepcopy(sol)
    
    return new_sol

def swap_schedules_partial(sol: dict, round: int, team_1: int, team_2: int):
    """
    Docstring
    """
    
    new_sol = copy.deepcopy(sol)
    
    return new_sol

def kempe_chain_move(sol: dict, team: int, round_1: int, round_2: int):
    """
    Docstring
    """
    
    new_sol = copy.deepcopy(sol)
    
    return new_sol


In [10]:
# Tests

n = 4
initial_sol = greedy_algorithm(n)
print("Initial solution:", initial_sol, "\n")

new_sol = swap_homes(initial_sol, 0, 1)
print("Swap homes:", new_sol)

new_sol = swap_rounds(initial_sol, 0, 1)
print("Swap rounds:", new_sol)

new_sol = swap_schedules(initial_sol, 0, 1)
print("Swap schedules:", new_sol)

new_sol = swap_rounds_partial(initial_sol, 0, 0, 1)
print("Swap rounds partial:", new_sol)

new_sol = swap_schedules_partial(initial_sol, 0, 0, 1)
print("Swap schedules partial:", new_sol)

new_sol = kempe_chain_move(initial_sol, 0, 0, 1)
print("Kempe chain move:", new_sol)

Initial solution: {0: [(0, 1), (2, 3)], 1: [(0, 2), (1, 3)], 2: [(0, 3), (1, 2)], 3: [(1, 0), (3, 2)], 4: [(2, 0), (3, 1)], 5: [(3, 0), (2, 1)]} 

Swap homes: {0: [(2, 3), (1, 0)], 1: [(0, 2), (1, 3)], 2: [(0, 3), (1, 2)], 3: [(3, 2), (0, 1)], 4: [(2, 0), (3, 1)], 5: [(3, 0), (2, 1)]}
Swap rounds: {0: [(0, 2), (1, 3)], 1: [(0, 1), (2, 3)], 2: [(0, 3), (1, 2)], 3: [(1, 0), (3, 2)], 4: [(2, 0), (3, 1)], 5: [(3, 0), (2, 1)]}
Swap schedules: {0: [(0, 1), (2, 3)], 1: [(1, 2), (0, 3)], 2: [(1, 3), (0, 2)], 3: [(1, 0), (3, 2)], 4: [(2, 1), (3, 0)], 5: [(3, 1), (2, 0)]}
Swap rounds partial: {0: [(0, 1), (2, 3)], 1: [(0, 2), (1, 3)], 2: [(0, 3), (1, 2)], 3: [(1, 0), (3, 2)], 4: [(2, 0), (3, 1)], 5: [(3, 0), (2, 1)]}
Swap schedules partial: {0: [(0, 1), (2, 3)], 1: [(0, 2), (1, 3)], 2: [(0, 3), (1, 2)], 3: [(1, 0), (3, 2)], 4: [(2, 0), (3, 1)], 5: [(3, 0), (2, 1)]}
Kempe chain move: {0: [(0, 1), (2, 3)], 1: [(0, 2), (1, 3)], 2: [(0, 3), (1, 2)], 3: [(1, 0), (3, 2)], 4: [(2, 0), (3, 1)], 5: [(3, 

In [ ]:
# Objective function

def calculate_total_distance(sol: dict, teams: dict, distances: dict):
    """
    Calculate total distance for feasible solutions.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - teams: dict --> dictionary of team homes
        - distances: dict --> dictionary of distances between team homes
    Output:
        - total_distance: float --> total distance travelled by all teams in the solution
    
    Note: this function assumes that the shortest path between two teams is the direct path between their homes
    """
    
    total_distance = 0
    for team in teams.keys():
        current_location = team
        for round in sol:
            for team_x, team_y in sol[round]:
                if team_y == team:
                    if current_location != team:
                        total_distance += distances[(current_location, team_x)]
                        current_location = team_x
                    else:
                        total_distance += distances[(team_y, team_x)]
                        current_location = team_x
                if (team_x == team) and (current_location != team):
                    total_distance += distances[(current_location, team_x)]
                    current_location = team_x
            if round == len(sol) - 1:
                total_distance += distances[(current_location, team)]   
    
    return total_distance

def violated_constraints(sol: dict, teams: dict, u: int = 3):
    """
    Checks how many times constraints of the TTP are violated in a solution (at-most, no-repeat).
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - teams: dict --> dictionary of team homes
        - u: int --> upper bound for the consecutive 'at home' or 'away' matches that a team is allowed to play
    Output:
        - num_violated_constraints: int --> number of times constraints are violated
    """
    
    consecutive_at_home = {i: 0 for i in range(len(teams))}
    consecutive_away = {i: 0 for i in range(len(teams))}
    num_violated_constraints = 0
    for round in range(len(sol) - 1):
        previous_matches = sol[round]
        for team_x, team_y in sol[round + 1]:
            if (team_y, team_x) in previous_matches:
                num_violated_constraints += 1
            if team_x in [same_team[0] for same_team in previous_matches]:
                consecutive_at_home[team_x] += 1
            if team_y in [same_team[0] for same_team in previous_matches]:
                consecutive_away[team_y] = 0
            if team_x in [same_team[1] for same_team in previous_matches]:
                consecutive_at_home[team_x] = 0
            if team_y in [same_team[1] for same_team in previous_matches]:
                consecutive_away[team_y] += 1
        if consecutive_at_home[team_x] >= u:
            num_violated_constraints += 1
        if consecutive_away[team_y] >= u:
            num_violated_constraints += 1
    
    return num_violated_constraints

def objective_function(sol: dict, teams: dict, distances: dict, w: float, func, u: int = 3):
    """
    Objective function for TTP data.
    Input:
        - sol: dict --> solution for double round-robin, where keys are rounds and values are lists of matchups
        - teams: dict --> dictionary of team homes
        - distances: dict --> dictionary of distances between team homes
        - w: float --> weight to control the impact of violated constraints
        - func: function --> function to control the impact of violated constraints
        - u: int --> upper bound for the consecutive 'at home' or 'away' matches that a team is allowed to play
    Output:
        - objective_value: float --> objective function's value
    """
    
    total_distance = calculate_total_distance(sol, teams, distances)
    num_violated_constraints = violated_constraints(sol, teams, u)
    
    if num_violated_constraints == 0:
        objective_value = total_distance
    else:
        objective_value = (total_distance + (w * func(num_violated_constraints))**2)**(1/2)
    
    return objective_value

In [21]:
# Tests

def constraint_penalty(num_violated_constraints: int):
    x = num_violated_constraints
    return 1 + (x**(1/2) + math.log(x)) / 2

n = 4
initial_sol = greedy_algorithm(n)
print("Initial solution:", initial_sol)

total_distance = calculate_total_distance(initial_sol, teams, distances)
print("Total distance:", total_distance)

num_violated_constraints = violated_constraints(initial_sol, teams)
print("Violated constraints:", num_violated_constraints)

objective_value = objective_function(initial_sol, teams, distances, 3000, constraint_penalty)
print("Objective value:", objective_value)

Initial solution: {0: [(0, 1), (2, 3)], 1: [(0, 2), (1, 3)], 2: [(0, 3), (1, 2)], 3: [(1, 0), (3, 2)], 4: [(2, 0), (3, 1)], 5: [(3, 0), (2, 1)]}
Total distance: 8682
Violated constraints: 0
Objective value: 8682


### Simulated Annealing heuristic

In [ ]:
# Optimization heuristic

def simulated_annealing(initial_sol: dict, objective_function_params: dict, temperature: float, cooling_rate: float, max_iter: int):
    """
    Simulated annealing algorithm for TTP data.
    Input:
        - initial_sol: dict --> initial solution for double round-robin, where keys are rounds and values are lists of matchups
        - distances: dict --> dictionary of distances between team homes
    Output:
        - final_sol: dict --> final solution for double round-robin (ideally, optimal or close to optimal for the TTP)
        - total_distance: float --> total distance travelled by all teams in the final solution
    """
    
    # Randomize initial solution
    #   - apply kempe move multiple times to initial solution
    
    # Run simulated annealing
    #   - calculate objective function for each neighbor
    #   - choose neighboring moves at random, with designed probabilities for each
    #   - if neighbor is better, accept it
    #   - if neighbor is worse, accept it with some probability that depends on the temperature
    #   - decrease temperature after multiple consecutive non-improving moves
    #   - repeat until maximum number of iterations is reached or temperature is too low
    
    # Return final solution
    
    return

### Integer Programming Formulation
gurobi ...